In [2]:
from bounos.Analyses import Weight
import operator
import pandas as pd
import itertools

_ = np.seterr(invalid='ignore') # Pandas PITA Nan printing

import bounos.ChartBuilders as cb
import bounos.Analyses.Trust as Trust

import os

def map_level(df, dct, level=0):
    index = df.index
    index.set_levels([[dct.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(index.levels)], inplace=True)


cb.latexify(columns=0.5, factor=1)

phys_keys = ['INDD','INHD','Speed']
comm_keys = ['ADelay','ARXP','ATXP','RXThroughput','TXThroughput','PLR']
observer = 'Bravo'
target = 'Alfa'
n_nodes = 6
n_metrics = 9
par=False

results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-07-20-17-47-53"

fig_basedir = "/home/bolster/src/thesis/papers/active/16_AAMAS/img"


var_rename_dict = {'CombinedBadMouthingPowerControl':'MPC',
 'CombinedSelfishTargetSelection':'STS',
 'CombinedTrust':'Fair',
 'Shadow':'Shadow',
 'SlowCoach':'SlowCoach'}

metric_rename_dict = {
    'ADelay': "$Delay$",
    'ARXP': "$P_{RX}$",
    'ATXP': "$P_{TX}$",
    'RXThroughput': "$T^P_{RX}$",
    'TXThroughput': "$T^P_{TX}$",
    'PLR': '$PLR$',
    'INDD': '$INDD$',
    'INHD': '$INHD$',
    'Speed': '$Speed$'
}
inv_map = lambda d: {v: k for k, v in d.items()}

results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-07-20-17-47-53"
with pd.get_store(results_path + "/outliers.bkup.h5") as store:
    print store.keys()
    weight_df = store.get(store.keys()[0])
    keys  = list(weight_df.keys())
    top_line = weight_df.values[0:100]
    del weight_df
    
from scipy.sparse import csc_matrix
def sparsify(a):
    indices = np.nonzero(~np.isnan(a))
    return csc_matrix((a[indices], indices), shape=a.shape, dtype=np.float64)

['/CombinedTrust0_3', '/CombinedTrust1_3', '/CombinedTrust2_3']


In [80]:
with pd.get_store(results_path + "/outliers.bkup.h5") as store:       
    mdts = store.get('meandeltaCombinedTrust_2')
np_mdts = np.split(mdts.values,[3], axis=1)
metrics = np.split(mdts.keys(),[4])[1]

from IPython.display import display
tops = {}
for k,v in mdts.groupby(['var']):
    tops[k]=v.sort(0, ascending=False)[metrics].head(1).values[0].astype(np.float64)

opti = pd.DataFrame.from_dict(tops, orient='index')
opti.columns = metrics
opti

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,ADelay,ARXP,ATXP,RXThroughput,PLR,TXThroughput,INDD,INHD,Speed
CombinedTrust,0,-0.5,0,0.0,0,0,0,0.5,0.0
Shadow,0,-1.0,0,0.0,0,0,0,0.0,0.0
CombinedBadMouthingPowerControl,0,0.0,1,0.0,0,0,0,0.0,0.0
SlowCoach,0,0.0,0,0.0,0,0,0,0.0,-1.0
CombinedSelfishTargetSelection,0,0.0,0,0.5,0,0,0,0.0,0.5


In [3]:
range_index = keys.index('range')

In [ ]:
configs_k, nodes_k, metrics_k = np.split(keys, (5, 5+n_nodes))
nodes_k

In [ ]:
configs, nodes, metrics = np.split(top_line, (5, 5+n_nodes), axis=1)
nodes=nodes.astype(np.float)
metrics=metrics.astype(np.float)
nodes[configs[:,-1]=='lower'] = -nodes[configs[:,-1]=='lower'

In [ ]:

sparsify(nodes).toarray().sum(axis=1)==np.nansum(nodes, axis=1)

In [ ]:
with pd.get_store(results_path + '.h5') as store:
    trust_observations = store.trust.xs('Bravo',level='observer', drop_level=False).dropna()
    
unweighted_trust_perspectives = Trust.generate_node_trust_perspective(trust_observations,
                                                                    metric_weights=None,
                                                                    par=False)
map_level(unweighted_trust_perspectives, var_rename_dict)

In [ ]:

def generate_outlier_tp(tf, sigma=None, good='good', good_lvl='bev'):
    """
    This must be applied on a per-observation basis (i.e. single run, single observer)
    :param tf:
    :param sigma:
    :param good:
    :param good_lvl:
    :return:
    """
    if sigma is not None:
        raise NotImplementedError("!Have't implemented")

    _mean=tf.xs(good,level=good_lvl).mean()
    _std=tf.xs(good,level=good_lvl).std()
    llim=_mean-_std
    ulim=_mean+_std
    uppers = (tf[tf>ulim]-ulim).reset_index()
    lowers = (llim-tf[tf<llim]).reset_index()
    outliers = pd.concat((uppers,lowers), keys=('upper','lower'), names=['range','id']).reset_index().set_index(
        ['var','run','observer','range','t']
    )
    outliers.drop('id', axis=1, inplace=True)
    return outliers

w = Weight.norm_weight(pd.Series(1,trust_observations.keys()))

l_outliers = []
for i, tf in unweighted_trust_perspectives.groupby(level=['run', 'observer']):
    l_outliers.append(generate_outlier_tp(tf, good='Fair', good_lvl='var'))
outlier = pd.concat(l_outliers).sort_index().dropna(how='all').reset_index()
for k in w.keys():
    outlier[k] = w[k]
outlier.set_index(['var', 't'], inplace=True)

In [ ]:
%matplotlib inline
weight_df = pd.concat([outlier]).reset_index()

observer = 'Bravo'
target = 'Alfa'
n_metrics = 9
# Select Perspective here
weight_df = weight_df[weight_df.observer == observer]
#weight_df = categorise_dataframe(weight_df)

# Metrics are the last sector of the frame, set these as leading indices
metric_keys = list(weight_df.keys()[-n_metrics:])
weight_df.set_index(metric_keys + ['var', 't'], inplace=True)

# REMEMBER TO CHECK THIS WHEN DOING MULTIPLE RUNS (although technically it shouldn't matter....)
weight_df.drop(['observer', 'run'], axis=1, inplace=True)

# Sum for each run (i.e. group by everything but time)
time_summed_weights = weight_df.groupby(level=list(weight_df.index.names[:-1])).sum().unstack('var')


In [ ]:
%%prun
d = {'upper':1, 'lower':-1}
r = weight_df['range'].apply(d.get)
f = lambda c: c*r
x=weight_df.drop('range', axis=1).apply(f)

In [ ]:
x.groupby(level=list(weight_df.index.names[:-1])).sum().unstack('var')

In [ ]:
#%%timeit
time_summed_signs = weight_df.apply(lambda v: -v[1:] if v[0][0]=='l' else v[1:], axis=1).groupby(level=list(weight_df.index.names[:-1])).sum().unstack('var')
time_summed_signs

In [ ]:
%%timeit
# Sum for each run (i.e. group by everything but time)
time_summed_weights = weight_df.groupby(level=list(weight_df.index.names[:-1])).sum().unstack('var')


In [ ]:
test = {'key':'val'}

def printer(**kwargs):
    print kwargs 
test.update({'another':'val'})
printer(**test)

In [17]:
(567.0/126400)*(4**9 - 3**9)/60

18.12702887658228

In [9]:
map(lambda x: 2^x, range(16))

[2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13]

Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 267, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 483, in getmodule
    file = getabsfile(object, _fil

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable